
### The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2020 Semester 1

## Assignment 1: Naive Bayes Classifiers

###### Submission deadline: 7 pm, Monday 20 Apr 2020

**Student Name(s):**    `PLEASE ENTER YOUR NAME(S) HERE`

**Student ID(s):**     `PLEASE ENTER YOUR ID(S) HERE`


This iPython notebook is a template which you will use for your Assignment 1 submission.

Marking will be applied on the four functions that are defined in this notebook, and to your responses to the questions at the end of this notebook (Submitted in a separate PDF file).

**NOTE: YOU SHOULD ADD YOUR RESULTS, DIAGRAMS AND IMAGES FROM YOUR OBSERVATIONS IN THIS FILE TO YOUR REPORT (the PDF file).**

You may change the prototypes of these functions, and you may write other functions, according to your requirements. We would appreciate it if the required functions were prominent/easy to find.

**Adding proper comments to your code is MANDATORY. **

In [34]:
import pandas as pd
import numpy as np
import math
from scipy import stats
from copy import copy, deepcopy

In [35]:
#    read all datasets 
adult = pd.read_csv("datasets/adult.data", header = None)
bank = pd.read_csv("datasets/bank.data", header = None)
breast_cancer_wisconsin = pd.read_csv("datasets/breast-cancer-wisconsin.data", header = None)
car = pd.read_csv("datasets/car.data", header = None)
lymphography = pd.read_csv("datasets/lymphography.data", header = None)
mushroom = pd.read_csv("datasets/mushroom.data", header = None)
nursery = pd.read_csv("datasets/nursery.data", header = None)
somerville = pd.read_csv("datasets/somerville.data", header = None)
university = pd.read_csv("datasets/university.data", header = None)
wdbc = pd.read_csv("datasets/wdbc.data", header = None)
wine = pd.read_csv("datasets/wine.data", header = None)

In [36]:
#   Helper function that delete all missing values
def delete_missing_value(raw_dataset, missing_values):
    rows = set(raw_dataset[raw_dataset.values == missing_values].index)
    data = raw_dataset.drop(index = rows)
    return data

In [37]:

#    This function should prepare the data by reading it from a file and
#    converting it into a useful format for training and testing

def preprocess(df, missing_values):
#     train=df.sample(frac=0.8,random_state=200)
#     test=df.drop(train.index)
#     Delete all missing values
    train = delete_missing_value(df, missing_values)
    return train


In [59]:
# Global Variable

#  Enter dataset here
missing_values = None
train_data = preprocess(wine, missing_values)

'''
    datasets_types can be 
    NOMINAL: NOMINAL ATTRIBUTES DATASETS, 
    NUMERIC: NUMERIC ATTRIBUTES DATASETS, 
    ORDINAL: ORDINAL ATTRIBUTES DATASETS, 
    MIX: DATASETS WITH A MIX OF ATTRIBUTE TYPES
'''
datasets_types = "NUMERIC"

# 0 represents nomianl, 1 represents ordinal, 2 represents numeric
feature_types = [2, 0, 2, 1, 1, 0, 0, 0, 0, 0, 2, 2, 2, 0]
class_column = 0
EPSILON = 1/(2*train_data.shape[0])

# 分开label和数据
label = train_data.iloc[:,class_column]
train_set = train_data.drop(columns=class_column)

feature_rows = list(train_set.columns)

print("row",feature_rows)
# Show some useful information about the data
print("Data Shape =====>", train_data.shape)
print("======================================================")
print("EPSILON ========>", EPSILON)
print("======================================================")
print("Some Data Examples:")
display(train_data.head(3))
print("======================================================")
print("Data Type:")
display(train_data.dtypes)

row [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
Data Shape =====> (178, 14)
EPSILON ========> 0.0028089887640449437
Some Data Examples:


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185


Data Type:


0       int64
1     float64
2     float64
3     float64
4     float64
5       int64
6     float64
7     float64
8     float64
9     float64
10    float64
11    float64
12    float64
13      int64
dtype: object

In [60]:
#calculate prior
def find_nominal_prob(class_column):
    prior = {}
    total_number = len(class_column)
    classes = class_column.value_counts()
    for key in classes.keys():
        prior[key] = math.log2(classes[key]/total_number)
    return prior

#Tutor版本:
def nominal_prob(series,denom,n_keys,k=0): 
    prior = {}
    counts = series.value_counts()
    keys = list(counts.keys())
    for key in keys: 
        prior[key] = np.log2((counts[key]+k)/(denom+k*n_keys)) 
    return prior 


In [61]:
# Calculate prior
print(find_nominal_prob(adult[class_column]))
print()
print("Tutor的:")
print("======================================================")
print(nominal_prob(label,train_data.shape[0],14,k=0))

{36: -5.18028575254781, 31: -5.196441520938851, 34: -5.1996944987339395, 23: -5.214424354844224, 35: -5.216070327687784, 33: -5.217718180569267, 28: -5.230969204067122, 30: -5.240987959892114, 37: -5.246023549789411, 25: -5.274895397033813, 27: -5.285224999927543, 32: -5.297370429789632, 38: -5.299113868117683, 39: -5.318432045317462, 29: -5.32374584521393, 41: -5.332645904537163, 24: -5.3506124510666115, 40: -5.357862190145999, 26: -5.374308543509969, 42: -5.383527073539347, 43: -5.402142751706694, 22: -5.411541449708944, 20: -5.434351332617029, 46: -5.465336578193888, 45: -5.471221134445745, 44: -5.491011500205753, 21: -5.499004290959283, 19: -5.51512395632256, 47: -5.52325183720596, 50: -5.757237710529256, 51: -5.774111529093652, 49: -5.818429878643475, 18: -5.887569578876936, 48: -5.906048999484596, 52: -6.089990579308209, 53: -6.132876392161386, 55: -6.280050953589606, 54: -6.29388986105467, 17: -6.365148544224493, 58: -6.475157549004915, 56: -6.475157549004915, 57: -6.50704161002

In [62]:
def find_nominal_llh(attributes, llh_dict, data, feature_row):
    all_feature = data[feature_row].value_counts()
    for attribute in attributes:
        feature_dict = {feature_row:{}}
        subset = data[data[class_column] == attribute][feature_row]
        feature_dict[feature_row] = find_nominal_prob(subset)
        for check_feature in all_feature.keys():
            if check_feature not in feature_dict[feature_row].keys():
                feature_dict[feature_row][check_feature] = np.log2(EPSILON)
        if(llh_dict[attribute]):
            llh_dict[attribute].update(feature_dict)
        else:
            llh_dict[attribute] = feature_dict
            
    return llh_dict

In [63]:
def find_numerical_llh(attributes, llh_dict, data, feature):
#  Save mean and Std as llh
    for attribute in attributes:
        feature_dict = {feature:{}}
        subset = subset = data[data[class_column] == attribute][feature]
        mean = np.mean(subset)
        std = np.std(subset)
        feature_dict[feature]["mean"] = mean
        feature_dict[feature]["std"] = std
        if(llh_dict[attribute]):
            llh_dict[attribute].update(feature_dict)
        else:
            llh_dict[attribute] = feature_dict
    return llh_dict

In [64]:
#  This function should calculat prior probabilities and likelihoods from the training data
#  and using them to build a naive Bayes model

def train(data):
    prior = find_nominal_prob(data[class_column])
    llh_dict = dict.fromkeys(prior)
    for i in feature_rows:
        if(datasets_types == "NUMERIC" or (datasets_types == "MIX" and feature_types[i] == 2)):
            find_numerical_llh(prior, llh_dict, data, i)
        else:
            find_nominal_llh(prior, llh_dict, data, i)
            
    return prior, llh_dict

prior, llh_dict = train(train_data)


In [65]:
# This function should predict classes for new items in a test dataset (for the purposes of this assignment, you
# can re-use the training data as a test set)

def predict(instance, prior, llh_dict):
    all_prob = {}
    for attribute in prior.keys():
        all_prob[attribute] = prior[attribute]
        for i in feature_rows:
            prob = 0
            if(datasets_types == "NUMERIC" or (datasets_types == "MIX" and feature_types[i] == 2)):
                mean = llh_dict[attribute][i]["mean"]
                std = llh_dict[attribute][i]["std"]
                prob = stats.norm.pdf(x=instance[i], loc=mean, scale=std)
                if prob > 0.0:
                    prob = np.log2(prob)
                else:
                    prob = np.log2(1 * 10 ** -8)
            else:
                prob = llh_dict[attribute][i][instance[i]]
            all_prob[attribute] += prob
    max_prob = -10000
    max_key = ""
    
    for prob in all_prob.keys():
        if(all_prob[prob] > max_prob):
            max_prob = all_prob[prob]
            max_key = prob
    return max_key

In [66]:
predict(train_data.iloc[100], prior, llh_dict)

2

In [67]:
# This function should evaliate the prediction performance by comparing your model’s class outputs to ground
# truth labels

def evaluate(data):
    predict_list = []
    correct = 0
    total = data.shape[0]
    for index, row in data.iterrows():
        if row[class_column] == predict(row, prior, llh_dict):
            correct += 1
#         result_list = [index,row[14],predict(row, prior, llh_dict)]
#         predict_list.append(result_list)
    correct_rate = correct/total
    return correct_rate
%time
print("======================================================")
print("Overall Accuracy is: ", evaluate(train_data))


Wall time: 0 ns
Overall Accuracy is:  0.9887640449438202


## Questions 


If you are in a group of 1, you will respond to question (1), and **one** other of your choosing (two responses in total).

If you are in a group of 2, you will respond to question (1) and question (2), and **two** others of your choosing (four responses in total). 

A response to a question should take about 100–250 words, and make reference to the data wherever possible.

#### NOTE: you may develope codes or functions in respond to the question, but your formal answer should be added to a separate file.

### Q1
Try discretising the numeric attributes in these datasets and treating them as discrete variables in the na¨ıve Bayes classifier. You can use a discretisation method of your choice and group the numeric values into any number of levels (but around 3 to 5 levels would probably be a good starting point). Does discretizing the variables improve classification performance, compared to the Gaussian na¨ıve Bayes approach? Why or why not?

In [113]:
#给原有的数据做一个备份
# backup = train_data.copy()

In [115]:
# #Change feature values
# feature = train_data.iloc[:,0]
# series = feature.copy()
# # series.sort_values()[0:30000]

In [116]:
# 找到这30000个数据,并把其的index换为临时index
# series[feature.sort_values()[0:30000].index] = 'a'
# series.value_counts()

In [173]:
# 两种办法: Equal Width 和 Equal Frequency

# 0 represents nomianl, 1 represents ordinal, 2 represents numeric
# feature_types=[2,0,2,1,1,0,0,0,0,0,2,2,2,0] 
def discretise(data,data_type,num_levels,by):
    '''
   给定一个数据集及其对应的数据类型，
   必须将其所有数字特征离散为离散特征，
   '宽度' & '频率'
   代表等宽或者等频率并且取一个可靠的整数，
   然后确定离散后的特征级别的总数
    '''
    copy_data = train_data.copy()
    discrete_datatype = feature_types.copy()
    
    for i in range(len(feature_types)):
        #找出所有的 Numeric Data,然后组成新的DataFrame
        # Note: 2 means numeric data type
        if feature_types[i] == 2:  
            new_dataFrame = data.iloc[:,i]
            copy_dataFrame = new_dataFrame.copy()
        
    # Equal Width
    if by=='width': 
    # calculate width of each level 
        maximum = np.max(new_dataFrame)
        minimum = np.min(new_dataFrame)
        width = (maximum-minimum)/num_levels # width of each level 
        for j in range(num_levels): 
            copy_feature[new_dataFrame<=(maximum-width*j)] = ('num_levels'+str(num_levels-j)) 

    # Equal Frequency 
    else:
        frequency = new_dataFrame.shape[0]/num_levels # frequency of each level 
        for m in range(num_levels): 
            lower = int(frequency*m) # lower bound of index, so has to be integer
            upper = int(frequency*(m+1)) # upper bound of index, so has to be integer 
            copy_feature[new_dataFrame.sort_values()[lower:upper].index] = ('num_levels' + str(num_levels-m))
        copy_data.iloc[:,i] = copy_feature 
        discrete_datatype[i] = 1
    return copy_data, discrete_datatype 

width_data,discrete_datatype = discretise(train_data,feature_types,num_levels=5,by='width')
freq_data,discrete_datatype = discretise(train_data,data_type,levels=5,by='freq')

NameError: name 'copy_feature' is not defined

In [170]:
width_data,discrete_datatype = discretise(train_data,feature_types,levels=5,by='width')
freq_data,discrete_datatype = discretise(train_data,data_type,levels=5,by='freq')
print(discrete_datatype)
print("======================================================")
print("Equal Width:")
display(width_data.iloc[:,10].value_counts())
print("======================================================")
print("Equal Frequency:")
display(freq_data.iloc[:,10].value_counts())

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Equal Width:


level1    29930
level5      148
level2       82
level3        2
Name: 10, dtype: int64

Equal Frequency:


level3    6033
level1    6033
level2    6032
level5    6032
level4    6032
Name: 10, dtype: int64

In [167]:
width_model = train(width_data)
width_preds = predict(width_data,prior,llh_dict)
width_eval = evaluate(width_data)


TypeError: Could not convert ... to numeric

In [171]:
freq_model = train(freq_data)
freq_preds = predict(freq_data,prior,llh_dict)
freq_eval = evaluate(freq_data)

TypeError: Could not convert ... to numeric

### Q2
Implement a baseline model (e.g., random or 0R) and compare the performance of the na¨ıve Bayes classifier to this baseline on multiple datasets. Discuss why the baseline performance varies across datasets, and to what extent the na¨ıve Bayes classifier improves on the baseline performance.

In [178]:
from random import randint
from random import seed

labels = list(train_data[class_column].value_counts().keys())
def random_pred(data,class_column): 
    predictions = []
    for i in range(data.shape[0]):
        instance = data.iloc[i,:]
        true_label = instance[class_column]
        seed(i) # different seed for each instance
        index = randint(0,len(labels)-1)
        pred = labels[index]
        predictions.append((i,true_label,pred))
    return predictions

random_predictions = random_pred(train_data,class_column)
random_eval = evaluate(random_predictions)

AttributeError: 'list' object has no attribute 'shape'

In [ ]:
def zero_r_pred(data,label_index):
    majority = data[label_index].value_counts().idxmax()
    predictions = []
    for i in range(data.shape[0]):
        instance = data.iloc[i,:]
        true_label = instance[label_index]
        predictions.append((i,true_label,majority))
    return predictions
zero_r_predictions = zero_r_pred(data,label_index)
zero_r_eval = evaluate(zero_r_predictions,labels)

### Q3
Since it’s difficult to model the probabilities of ordinal data, ordinal attributes are often treated as either nominal variables or numeric variables. Compare these strategies on the ordinal datasets provided. Deterimine which approach gives higher classification accuracy and discuss why.

### Q4
Evaluating the model on the same data that we use to train the model is considered to be a major mistake in Machine Learning. Implement a hold–out or cross–validation evaluation strategy (you should implement this yourself and do not simply call existing implementations from `scikit-learn`). How does your estimate of effectiveness change, compared to testing on the training data? Explain why. (The result might surprise you!)

### Q5
Implement one of the advanced smoothing regimes (add-k, Good-Turing). Does changing the smoothing regime (or indeed, not smoothing at all) affect the effectiveness of the na¨ıve Bayes classifier? Explain why, or why not.

### Q6
The Gaussian na¨ıve Bayes classifier assumes that numeric attributes come from a Gaussian distribution. Is this assumption always true for the numeric attributes in these datasets? Identify some cases where the Gaussian assumption is violated and describe any evidence (or lack thereof) that this has some effect on the NB classifier’s predictions.